In [ ]:
#定义在装箱问题下的粒子群算法的适应度函数
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
data = pd.read_excel('/Users/Documents/GitHub/mathematical-modeling/11/附件1-装箱数据.xlsx',sheet_name=None)

pakage_data = data['耗材数据']
express_data = data['订单数据']
data_count = data['count']
express_data_list = express_data.values.tolist()
print(len(express_data_list))


In [ ]:

from decimal import Decimal
DEFAULT_NUMBER_OF_DECIMALS = 3
START_POSITION = [0, 0, 0]
def get_limit_number_of_decimals(number_of_decimals):
    return Decimal('1.{}'.format('0' * number_of_decimals))

def set_to_decimal(value, number_of_decimals):
    number_of_decimals = get_limit_number_of_decimals(number_of_decimals)

    return Decimal(value).quantize(number_of_decimals)


def rect_intersect(item1, item2, x, y):
    """Estimate whether two items get intersection in one dimension.
    Args:
        item1, item2: any two items in item list.
        x,y: Axis.LENGTH/ Axis.Height/ Axis.WIDTH.
    Returns:
        Boolean variable: False when two items get intersection in one dimension; True when two items do not intersect in one dimension.
    """

    d1 = item1.get_dimension()
    d2 = item2.get_dimension()

    cx1 = item1.position[x] + d1[x] / 2
    cy1 = item1.position[y] + d1[y] / 2
    cx2 = item2.position[x] + d2[x] / 2
    cy2 = item2.position[y] + d2[y] / 2

    ix = max(cx1, cx2) - min(cx1, cx2)  # ix: |cx1-cx2|
    iy = max(cy1, cy2) - min(cy1, cy2)  # iy: |cy1-cy2|

    return ix < (d1[x] + d2[x]) / 2 and iy < (d1[y] + d2[y]) / 2


def rect_intersect(item1, item2, x, y):
    """Estimate whether two items get intersection in one dimension.
    Args:
        item1, item2: any two items in item list.
        x,y: Axis.LENGTH/ Axis.Height/ Axis.WIDTH.
    Returns:
        Boolean variable: False when two items get intersection in one dimension; True when two items do not intersect in one dimension.
    """

    d1 = item1.get_dimension()
    d2 = item2.get_dimension()

    cx1 = item1.position[x] + d1[x] / 2
    cy1 = item1.position[y] + d1[y] / 2
    cx2 = item2.position[x] + d2[x] / 2
    cy2 = item2.position[y] + d2[y] / 2

    ix = max(cx1, cx2) - min(cx1, cx2)  # ix: |cx1-cx2|
    iy = max(cy1, cy2) - min(cy1, cy2)  # iy: |cy1-cy2|

    return ix < (d1[x] + d2[x]) / 2 and iy < (d1[y] + d2[y]) / 2


def stack(item1, item2):
    """Stack two items with same length, width, height or any two of three sides are same.
    Args:
        item1, item2: any two items in item list.
    Return:
        item1/ stacked_item_list/ stacked_item.
    """

    if (
            item1.length == item2.length and
            item1.width == item2.width and
            item1.height == item2.height
    ):
        stacked_item_1 = Item(item1.name + item2.name, item1.length + item2.length,
                              item1.width, item1.height, item1.weight + item2.weight)  # (2l, w, h)
        stacked_item_2 = Item(item1.name + item2.name, item1.length, item1.width + item2.width,
                              item1.height, item1.weight + item2.weight)  # (l, 2w, h)
        stacked_item_3 = Item(item1.name + item2.name, item1.length, item1.width,
                              item1.height + item2.height, item1.weight + item2.weight)  # (l, w, 2h)

        stacked_item_list = [stacked_item_1, stacked_item_2, stacked_item_3]

        return stacked_item_list

    elif (
            item1.length == item2.length and
            item1.width == item2.width and
            item1.height != item2.height
    ):
        stacked_item = Item(item1.name + item2.name, item1.length, item1.width,
                            item1.height + item2.height, item1.weight + item2.weight)  # (l, w, 2h)

        return stacked_item

    elif (
            item1.length == item2.length and
            item1.height == item2.height and
            item1.width != item2.width
    ):
        stacked_item = Item(item1.name + item2.name, item1.length, item1.width + item2.width,
                            item1.height, item1.weight + item2.weight)  # (l, 2w, h)

        return stacked_item

    elif (
            item1.width == item2.width and
            item1.height == item2.height and
            item1.length != item2.length
    ):
        stacked_item = Item(item1.name + item2.name, item1.length + item2.length,
                            item1.width, item1.height, item1.weight + item2.weight)

        return stacked_item  # (2l, w, h)

    else:
        return item1


def intersect(item1, item2):
    """Estimate whether two items get intersection in 3D dimension.
    Args:
        item1, item2: any two items in item list.
    Returns:
        Boolean variable: False when two items get intersection; True when two items do not intersect.
    """

    return (
            rect_intersect(item1, item2, Axis.LENGTH, Axis.HEIGHT) and  # xz dimension
            rect_intersect(item1, item2, Axis.HEIGHT, Axis.WIDTH) and  # yz dimension
            rect_intersect(item1, item2, Axis.LENGTH, Axis.WIDTH))  # xy dimension

class RotationType:
    RT_LWH = 0
    RT_HLW = 1
    RT_HWL = 2
    RT_WHL = 3
    RT_WLH = 4
    RT_LHW = 5

    ALL = [RT_LWH, RT_HLW, RT_HWL, RT_WHL, RT_WLH, RT_LHW]

class Axis:
    LENGTH = 0
    WIDTH = 1
    HEIGHT = 2

    ALL = [LENGTH, WIDTH, HEIGHT]

class Item:
    def __init__(self, name, length, width, height, weight):
        self.name = name
        self.length = length
        self.width = width
        self.height = height
        self.weight = weight
        self.rotation_type = 0  # initial rotation type: (x, y, z) --> (l, w, h)
        self.position = START_POSITION  # initial position: (0, 0, 0)
        self.number_of_decimals = DEFAULT_NUMBER_OF_DECIMALS

    def format_numbers(self, number_of_decimals):
        self.length = set_to_decimal(self.length, number_of_decimals)
        self.height = set_to_decimal(self.height, number_of_decimals)
        self.width = set_to_decimal(self.width, number_of_decimals)
        self.weight = set_to_decimal(self.weight, number_of_decimals)
        self.number_of_decimals = number_of_decimals

    def get_volume(self):
        return set_to_decimal(
            self.length * self.height * self.width, self.number_of_decimals)

    def get_dimension(self):  # 6 orientation types -- (x-axis, y-axis, z-axis)
        if self.rotation_type == RotationType.RT_LWH:
            dimension = [self.length, self.width, self.height]
        elif self.rotation_type == RotationType.RT_HLW:
            dimension = [self.height, self.length, self.width]
        elif self.rotation_type == RotationType.RT_HWL:
            dimension = [self.height, self.width, self.length]
        elif self.rotation_type == RotationType.RT_WHL:
            dimension = [self.width, self.height, self.length]
        elif self.rotation_type == RotationType.RT_WLH:
            dimension = [self.width, self.length, self.height]
        elif self.rotation_type == RotationType.RT_LHW:
            dimension = [self.length, self.height, self.width]
        else:
            dimension = []

        return dimension

    def string(self):
        return "%s(%sx%sx%s, weight: %s) pos(%s) rt(%s) vol(%s)" % (
            self.name, self.length, self.width, self.height, self.weight,
            self.position, self.rotation_type, self.get_volume()
        )
    def print_list(self):
        return [self.name, self.length, self.width, self.height, 0]

class Packer:
    def __init__(self):
        self.bins = []
        self.unplaced_items = []
        self.placed_items = []
        self.unfit_items = []
        self.total_items = 0
        self.total_used_bins = 0  # not used yet
        self.used_bins = []  # not used yet

    def add_bin(self, bin):
        return self.bins.append(bin)

    def add_item(self, item):
        """Add unplaced items into bin's unplaced_items list.
        Args:
            item: an unplaced item.
        Returns:
            The unplaced item is added into bin's unplaced_items list."""

        self.total_items += 1
        return self.unplaced_items.append(item)

    def pivot_dict(self, bin, item):
        """For each item to be placed into a certain bin, obtain a corresponding comparison parameter of each optional pivot that the item can be placed.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            a pivot_dict contain all optional pivot point and their comparison parameter of the item.
            a empty dict may be returned if the item couldn't be placed into the bin.
        """

        pivot_dict = {}
        can_put = False

        for axis in range(0, 3):
            items_in_bin = bin.items
            items_in_bin_temp = items_in_bin[:]

            n = 0
            while n < len(items_in_bin):
                pivot = [0, 0, 0]

                if axis == Axis.LENGTH:  # axis = 0/ x-axis
                    ib = items_in_bin[n]
                    pivot = [ib.position[0] + ib.get_dimension()[0],
                             ib.position[1],
                             ib.position[2]]
                    try_put_item = bin.can_hold_item_with_rotation(item, pivot)

                    if try_put_item:
                        can_put = True
                        q = 0
                        q = 0
                        ib_neigh_x_axis = []
                        ib_neigh_y_axis = []
                        ib_neigh_z_axis = []
                        right_neighbor = False
                        front_neighbor = False
                        above_neighbor = False

                        while q < len(items_in_bin_temp):
                            if items_in_bin_temp[q] == items_in_bin[n]:
                                q += 1

                            else:
                                ib_neighbor = items_in_bin_temp[q]

                                if (
                                        ib_neighbor.position[0] > ib.position[0] + ib.get_dimension()[0] and
                                        ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] and
                                        ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2]
                                ):
                                    right_neighbor = True
                                    x_distance = ib_neighbor.position[0] - (ib.position[0] + ib.get_dimension()[0])
                                    ib_neigh_x_axis.append(x_distance)

                                elif (
                                        ib_neighbor.position[1] >= ib.position[1] + ib.get_dimension()[1] and
                                        ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] +
                                        ib.get_dimension()[0] and
                                        ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2]
                                ):
                                    front_neighbor = True
                                    y_distance = ib_neighbor.position[1] - ib.position[1]
                                    ib_neigh_y_axis.append(y_distance)

                                elif (
                                        ib_neighbor.position[2] >= ib.position[2] + ib.get_dimension()[2] and
                                        ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] +
                                        ib.get_dimension()[0] and
                                        ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1]
                                ):
                                    above_neighbor = True
                                    z_distance = ib_neighbor.position[2] - ib.position[2]
                                    ib_neigh_z_axis.append(z_distance)

                                q += 1

                        if not right_neighbor:
                            x_distance = bin.length - (ib.position[0] + ib.get_dimension()[0])
                            ib_neigh_x_axis.append(x_distance)

                        if not front_neighbor:
                            y_distance = bin.width - ib.position[1]
                            ib_neigh_y_axis.append(y_distance)

                        if not above_neighbor:
                            z_distance = bin.height - ib.position[2]
                            ib_neigh_z_axis.append(z_distance)

                        distance_3D = [min(ib_neigh_x_axis), min(ib_neigh_y_axis), min(ib_neigh_z_axis)]
                        pivot_dict[tuple(pivot)] = distance_3D

                elif axis == Axis.WIDTH:  # axis = 1/ y-axis
                    ib = items_in_bin[n]
                    pivot = [ib.position[0],
                             ib.position[1] + ib.get_dimension()[1],
                             ib.position[2]]
                    try_put_item = bin.can_hold_item_with_rotation(item, pivot)

                    if try_put_item:
                        can_put = True
                        q = 0
                        ib_neigh_x_axis = []
                        ib_neigh_y_axis = []
                        ib_neigh_z_axis = []
                        right_neighbor = False
                        front_neighbor = False
                        above_neighbor = False

                        while q < len(items_in_bin_temp):
                            if items_in_bin_temp[q] == items_in_bin[n]:
                                q += 1

                            else:
                                ib_neighbor = items_in_bin_temp[q]

                                if (
                                        ib_neighbor.position[0] >= ib.position[0] + ib.get_dimension()[0] and
                                        ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] +
                                        ib.get_dimension()[1] and
                                        ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2]
                                ):
                                    right_neighbor = True
                                    x_distance = ib_neighbor.position[0] - ib.position[0]
                                    ib_neigh_x_axis.append(x_distance)

                                elif (
                                        ib_neighbor.position[1] > ib.position[1] + ib.get_dimension()[1] and
                                        ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] and
                                        ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2]
                                ):
                                    front_neighbor = True
                                    y_distance = ib_neighbor.position[1] - (ib.position[1] + ib.get_dimension()[1])
                                    ib_neigh_y_axis.append(y_distance)

                                elif (
                                        ib_neighbor.position[2] >= ib.position[2] + ib.get_dimension()[2] and
                                        ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] and
                                        ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] +
                                        ib.get_dimension()[1]
                                ):
                                    above_neighbor = True
                                    z_distance = ib_neighbor.position[2] - ib.position[2]
                                    ib_neigh_z_axis.append(z_distance)

                                q += 1

                        if not right_neighbor:
                            x_distance = bin.length - ib.position[0]
                            ib_neigh_x_axis.append(x_distance)

                        if not front_neighbor:
                            y_distance = bin.width - (ib.position[1] + ib.get_dimension()[1])
                            ib_neigh_y_axis.append(y_distance)

                        if not above_neighbor:
                            z_distance = bin.height - ib.position[2]
                            ib_neigh_z_axis.append(z_distance)

                        distance_3D = [min(ib_neigh_x_axis), min(ib_neigh_y_axis), min(ib_neigh_z_axis)]
                        pivot_dict[tuple(pivot)] = distance_3D

                elif axis == Axis.HEIGHT:  # axis = 2/ z-axis
                    ib = items_in_bin[n]
                    pivot = [ib.position[0],
                             ib.position[1],
                             ib.position[2] + ib.get_dimension()[2]]
                    try_put_item = bin.can_hold_item_with_rotation(item, pivot)

                    if try_put_item:
                        can_put = True
                        q = 0
                        ib_neigh_x_axis = []
                        ib_neigh_y_axis = []
                        ib_neigh_z_axis = []
                        right_neighbor = False
                        front_neighbor = False
                        above_neighbor = False

                        while q < len(items_in_bin_temp):
                            if items_in_bin_temp[q] == items_in_bin[n]:
                                q += 1

                            else:
                                ib_neighbor = items_in_bin_temp[q]

                                if (
                                        ib_neighbor.position[0] >= ib.position[0] + ib.get_dimension()[0] and
                                        ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] and
                                        ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] +
                                        ib.get_dimension()[2]
                                ):
                                    right_neighbor = True
                                    x_distance = ib_neighbor.position[0] - ib.position[0]
                                    ib_neigh_x_axis.append(x_distance)

                                elif (
                                        ib_neighbor.position[1] > ib.position[1] + ib.get_dimension()[1] and
                                        ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0] and
                                        ib_neighbor.position[2] + ib_neighbor.get_dimension()[2] > ib.position[2] +
                                        ib.get_dimension()[2]
                                ):
                                    front_neighbor = True
                                    y_distance = ib_neighbor.position[1] - (ib.position[1] + ib.get_dimension()[1])
                                    ib_neigh_y_axis.append(y_distance)

                                elif (
                                        ib_neighbor.position[2] >= ib.position[2] + ib.get_dimension()[2] and
                                        ib_neighbor.position[1] + ib_neighbor.get_dimension()[1] > ib.position[1] and
                                        ib_neighbor.position[0] + ib_neighbor.get_dimension()[0] > ib.position[0]
                                ):
                                    above_neighbor = True
                                    z_distance = ib_neighbor.position[2] - ib.position[2]
                                    ib_neigh_z_axis.append(z_distance)

                                q += 1

                        if not right_neighbor:
                            x_distance = bin.length - ib.position[0]
                            ib_neigh_x_axis.append(x_distance)

                        if not front_neighbor:
                            y_distance = bin.width - ib.position[1]
                            ib_neigh_y_axis.append(y_distance)

                        if not above_neighbor:
                            z_distance = bin.height - (ib.position[2] + ib.get_dimension()[2])
                            ib_neigh_z_axis.append(z_distance)

                        distance_3D = [min(ib_neigh_x_axis), min(ib_neigh_y_axis), min(ib_neigh_z_axis)]
                        pivot_dict[tuple(pivot)] = distance_3D

                n += 1

        return pivot_dict

    def pivot_list(self, bin, item):
        """Obtain all optional pivot points that one item could be placed into a certain bin.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            a pivot_list containing all optional pivot points that the item could be placed into a certain bin.
        """

        pivot_list = []

        for axis in range(0, 3):
            items_in_bin = bin.items

            for ib in items_in_bin:
                pivot = [0, 0, 0]
                if axis == Axis.LENGTH:  # axis = 0/ x-axis
                    pivot = [ib.position[0] + ib.get_dimension()[0],
                             ib.position[1],
                             ib.position[2]]
                elif axis == Axis.WIDTH:  # axis = 1/ y-axis
                    pivot = [ib.position[0],
                             ib.position[1] + ib.get_dimension()[1],
                             ib.position[2]]
                elif axis == Axis.HEIGHT:  # axis = 2/ z-axis
                    pivot = [ib.position[0],
                             ib.position[1],
                             ib.position[2] + ib.get_dimension()[2]]

                pivot_list.append(pivot)

        return pivot_list

    def choose_pivot_point(self, bin, item):
        """Choose the optimal one from all optional pivot points of the item after comparison.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            the optimal pivot point that a item could be placed into a bin.
        """

        can_put = False
        pivot_available = []
        pivot_available_temp = []
        vertex_3d = []
        vertex_2d = []
        vertex_1d = []

        n = 0
        m = 0
        p = 0

        pivot_list = self.pivot_list(bin, item)

        for pivot in pivot_list:
            try_put_item = bin.can_hold_item_with_rotation(item, pivot)

            if try_put_item:
                can_put = True
                pivot_available.append(pivot)
                pivot_temp = sorted(pivot)
                pivot_available_temp.append(pivot_temp)

        if pivot_available:
            while p < len(pivot_available_temp):
                vertex_3d.append(pivot_available_temp[p][0])
                p += 1

            p = 0
            while p < len(pivot_available_temp):
                if pivot_available_temp[p][0] == min(vertex_3d):
                    n += 1
                    vertex_2d.append(pivot_available_temp[p][1])

                p += 1

            if n == 1:
                p = 0
                while p < len(pivot_available_temp):
                    if pivot_available_temp[p][0] == min(pivot_available_temp[p]):
                        return pivot_available[p]

                    p += 1

            else:
                p = 0
                while p < len(pivot_available_temp):
                    if (
                            pivot_available_temp[p][0] == min(pivot_available_temp[p]) and
                            pivot_available_temp[p][1] == min(vertex_2d)
                    ):
                        m += 1
                        vertex_1d.append(pivot_available_temp[p][2])

                    p += 1

            if m == 1:
                p = 0
                while p < len(pivot_available_temp):
                    if (
                            pivot_available_temp[p][0] == min(pivot_available_temp[p]) and
                            pivot_available_temp[p][1] == min(vertex_2d)
                    ):
                        return pivot_available[p]

                    p += 1

            else:
                p = 0
                while p < len(pivot_available_temp):
                    if (
                            pivot_available_temp[p][0] == min(pivot_available_temp[p]) and
                            pivot_available_temp[p][1] == min(vertex_2d) and
                            pivot_available_temp[p][2] == min(vertex_1d)
                    ):
                        return pivot_available[p]

                    p += 1

        if not pivot_available:
            return can_put

    def pack_to_bin(self, bin, item):
        """For each item and each bin, perform whole pack process with optimal orientation and pivot point.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns: return value is void.
        """

        if not bin.items:
            response = bin.put_item(item, START_POSITION, [bin.length, bin.width, bin.height])

            if not response:
                bin.unfitted_items.append(item)

            return

        else:
            pivot_point = self.choose_pivot_point(bin, item)
            pivot_dict = self.pivot_dict(bin, item)

            if not pivot_point:
                bin.unfitted_items.append(item)
                return

            distance_3D = pivot_dict[tuple(pivot_point)]
            response = bin.put_item(item, pivot_point, distance_3D)
            return

    def pack(
            self, bigger_first=True, number_of_decimals=DEFAULT_NUMBER_OF_DECIMALS):
        """For a list of items and a list of bins, perform the whole pack process.
        Args:
            bin: a bin in bin list that a certain item will be placed into.
            item: an unplaced item in item list.
        Returns:
            For each bin, print detailed information about placed and unplaced items.
            Then, print the optimal bin with highest packing rate.
        """

        for bin in self.bins:
            bin.format_numbers(number_of_decimals)

        for unplaced_item in self.unplaced_items:
            unplaced_item.format_numbers(number_of_decimals)

        self.bins.sort(
            key=lambda bin: bin.get_volume())  # default order of bins: from smallest to biggest
        self.unplaced_items.sort(
            key=lambda unplaced_item: unplaced_item.get_volume(),
            reverse=bigger_first)  # default order of items: from biggest to smallest

        filling_ratio_list = []

        for bin in self.bins:
            for unplaced_item in self.unplaced_items:
                bin.unplaced_items.append(unplaced_item)

        for bin in self.bins:
            for unplaced_item in self.unplaced_items:
                self.pack_to_bin(bin, unplaced_item)

            # print("\n:::::::::::", bin.string())
            # print("FITTED ITEMS:")
            # for item in bin.items:
            #     print("====> ", item.string())
            #
            # print("\nUNFITTED ITEMS:")
            # for item in bin.unfitted_items:
            #     print("====> ", item.string())

            filling_ratio_list.append(bin.get_filling_ratio())

        max_filling_ratio = max(filling_ratio_list)
        unfit_num = []
        name_bin = 'none'
        for bin in self.bins:

            if bin.get_filling_ratio() == max_filling_ratio:
                for item in bin.items:
                    self.placed_items.append(item)
                # print("\nSelected bin with highest filling ratio: ", bin.string())
                name_bin = bin.print_name()
                for item in bin.unfitted_items:
                    unfit_num.append(item.print_list())
                return unfit_num,name_bin



class Bin:
    def __init__(self, size, length, width, height, capacity):
        self.size = size
        self.length = length
        self.width = width
        self.height = height
        self.capacity = capacity
        self.total_items = 0  # number of total items in one bin
        self.items = []  # item in one bin -- a blank list initially
        self.unplaced_items = []
        self.unfitted_items = []  # unfitted item in one bin -- a blank list initially
        self.number_of_decimals = DEFAULT_NUMBER_OF_DECIMALS

    def format_numbers(self, number_of_decimals):
        self.length = set_to_decimal(self.length, number_of_decimals)
        self.height = set_to_decimal(self.height, number_of_decimals)
        self.width = set_to_decimal(self.width, number_of_decimals)
        self.capacity = set_to_decimal(self.capacity, number_of_decimals)
        self.number_of_decimals = number_of_decimals

    def get_volume(self):
        return set_to_decimal(
            self.length * self.height * self.width, self.number_of_decimals)

    def get_total_weight(self):
        total_weight = 0

        for item in self.items:
            total_weight += item.weight

        return set_to_decimal(total_weight, self.number_of_decimals)

    def get_filling_ratio(self):
        total_filling_volume = 0
        total_filling_ratio = 0

        for item in self.items:
            total_filling_volume += item.get_volume()

        total_filling_ratio = total_filling_volume / self.get_volume()
        return set_to_decimal(total_filling_ratio, self.number_of_decimals)

# 添加一种情况，即有袋子装入的情况
    def can_hold_item_with_rotation(self, item, pivot):
        """Evaluate whether one item can be placed into bin with all optional orientations.
        Args:
            item: any item in item list.
            pivot: an (x, y, z) coordinate, the back-lower-left corner of the item will be placed at the pivot.
        Returns:
            a list containing all optional orientations. If not, return an empty list.
        """

        fit = False
        valid_item_position = [0, 0, 0]
        item.position = pivot
        rotation_type_list = []
        # if not 'Bag' in self.size  :
        for i in range(0, len(RotationType.ALL)):
            item.rotation_type = i
            dimension = item.get_dimension()
            if (
                    pivot[0] + dimension[0] <= self.length and  # x-axis
                    pivot[1] + dimension[1] <= self.width and  # y-axis
                    pivot[2] + dimension[2] <= self.height  # z-axis
            ):

                fit = True

                for current_item_in_bin in self.items:
                    if intersect(current_item_in_bin, item):
                        fit = False
                        item.position = [0, 0, 0]
                        break

                if fit:
                    if self.get_total_weight() + item.weight > self.capacity:  # estimate whether bin reaches its capacity
                        fit = False
                        item.position = [0, 0, 0]
                        continue

                    else:
                        rotation_type_list.append(item.rotation_type)

            else:
                continue
        # else:
        #     for i in range(0, len(RotationType.ALL)):
        #         item.rotation_type = i
        #         dimension = item.get_dimension()
        #         if (
        #                 (pivot[0]+pivot[2] + dimension[0]+dimension[2]<=self.length+self.height) and
        #                 (pivot[1]+pivot[2] + dimension[1]+dimension[2]<=self.width+self.height)
        #
        #
        #                 # pivot[0] + dimension[0] <= self.length and  # x-axis
        #                 # pivot[1] + dimension[1] <= self.width and  # y-axis
        #                 # pivot[2] + dimension[2] <= self.height  # z-axis
        #         ):
        #
        #             fit = True
        #
        #             for current_item_in_bin in self.items:
        #                 if intersect(current_item_in_bin, item):
        #                     fit = False
        #                     item.position = [0, 0, 0]
        #                     break
        #
        #             if fit:
        #                 if self.get_total_weight() + item.weight > self.capacity:  # estimate whether bin reaches its capacity
        #                     fit = False
        #                     item.position = [0, 0, 0]
        #                     continue
        #
        #                 else:
        #                     rotation_type_list.append(item.rotation_type)
        #
        #         else:
        #             continue

        return rotation_type_list

    def put_item(self, item, pivot, distance_3d):
        """Evaluate whether an item can be placed into a certain bin with which orientation. If yes, perform put action.
        Args:
            item: any item in item list.
            pivot: an (x, y, z) coordinate, the back-lower-left corner of the item will be placed at the pivot.
            distance_3d: a 3D parameter to determine which orientation should be chosen.
        Returns:
            Boolean variable: False when an item couldn't be placed into the bin; True when an item could be placed and perform put action.
        """

        fit = False
        rotation_type_list = self.can_hold_item_with_rotation(item, pivot)
        margins_3d_list = []
        margins_3d_list_temp = []
        margin_3d = []
        margin_2d = []
        margin_1d = []

        n = 0
        m = 0
        p = 0

        if not rotation_type_list:
            return fit

        else:
            fit = True

            rotation_type_number = len(rotation_type_list)

            if rotation_type_number == 1:
                item.rotation_type = rotation_type_list[0]
                self.items.append(item)
                self.total_items += 1
                return fit

            else:
                for rotation in rotation_type_list:
                    item.rotation_type = rotation
                    dimension = item.get_dimension()
                    margins_3d = [distance_3d[0] - dimension[0],
                                  distance_3d[1] - dimension[1],
                                  distance_3d[2] - dimension[2]]
                    margins_3d_temp = sorted(margins_3d)
                    margins_3d_list.append(margins_3d)
                    margins_3d_list_temp.append(margins_3d_temp)

                while p < len(margins_3d_list_temp):
                    margin_3d.append(margins_3d_list_temp[p][0])
                    p += 1

                p = 0
                while p < len(margins_3d_list_temp):
                    if margins_3d_list_temp[p][0] == min(margin_3d):
                        n += 1
                        margin_2d.append(margins_3d_list_temp[p][1])

                    p += 1

                if n == 1:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if margins_3d_list_temp[p][0] == min(margin_3d):
                            item.rotation_type = rotation_type_list[p]
                            self.items.append(item)
                            self.total_items += 1
                            return fit

                        p += 1

                else:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if (
                                margins_3d_list_temp[p][0] == min(margin_3d) and
                                margins_3d_list_temp[p][1] == min(margin_2d)
                        ):
                            m += 1
                            margin_1d.append(margins_3d_list_temp[p][2])

                        p += 1

                if m == 1:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if (
                                margins_3d_list_temp[p][0] == min(margin_3d) and
                                margins_3d_list_temp[p][1] == min(margin_2d)
                        ):
                            item.rotation_type = rotation_type_list[p]
                            self.items.append(item)
                            self.total_items += 1
                            return fit

                        p += 1

                else:
                    p = 0
                    while p < len(margins_3d_list_temp):
                        if (
                                margins_3d_list_temp[p][0] == min(margin_3d) and
                                margins_3d_list_temp[p][1] == min(margin_2d) and
                                margins_3d_list_temp[p][2] == min(margin_1d)
                        ):
                            item.rotation_type = rotation_type_list[p]
                            self.items.append(item)
                            self.total_items += 1
                            return fit

                        p += 1

    def string(self):
        return "%s(%sx%sx%s, max_weight:%s) vol(%s) item_number(%s) filling_ratio(%s)" % (
            self.size, self.length, self.width, self.height, self.capacity,
            self.get_volume(), self.total_items, self.get_filling_ratio())
    def print_name(self):
        return self.size
def check_volume(good,box):
    flag = False
    if (good[0]<box[0] and good[1]<box[1] and good[2]<box[2]) or (good[0]<box[0] and good[2]<box[1] and good[1]<box[2]) or(good[2]<box[0] and good[1]<box[1] and good[0]<box[2]) or (good[2]<box[0] and good[0]<box[1] and good[1]<box[2]) or (good[1]<box[0] and good[0]<box[1] and good[2]<box[2]) or (good[1]<box[0] and good[2]<box[1] and good[0]<box[2]):
        flag = True
    return flag

#计算出纸袋体积，与物体比较，然后算出体积大于的纸箱
Box_size = [
    [165,120,55],
    [200,140,70],
    [200,150,150],
    [270,200,90],
    [300,200,170]
]


Bag_size = [
    [179.3421433,119.3421433,71.6578567],
    [210.1253636,150.1253636,90.87463639],
    [280.662174,220.662174,120.337826],
    [305.9244646,245.9244646,145.0755354]
]
Bag_volume = [1533698.536,2866653.729,7452705.136,10914657.83,1089000,1960000,4500000,4860000,10200000]

In [ ]:
ca_more = express_data_list.copy()
import gc
process_Bag_Box_data = ca_more.copy()


def func(x):
    Box_size = [
    [165+x[0],120+x[1],55+x[2]],
    [200+x[3],140+x[4],70+x[5]],
    [200+x[6],150+x[7],150+x[8]],
    [270+x[9],200+x[10],90+x[11]],
    [300+x[12],200+x[13],170+x[14]]
]
    Bag_size = [
    [179.3421433+x[15],119.3421433+x[16],71.6578567+x[17]],
    [210.1253636+x[18],150.1253636+x[19],90.87463639+x[20]],
    [280.662174+x[21],220.662174+x[22],120.337826+x[23]],
    [305.9244646+x[24],245.9244646+x[25],145.0755354+x[26]]
]
    process_Bag_Box_data = ca_more.copy()
    for i in range(len(ca_more)):
        if not (check_volume(process_Bag_Box_data[len(ca_more)-1-i][1:4],Bag_size[3]) or(check_volume(process_Bag_Box_data[len(ca_more)-1-i][1:4],Box_size[4]))):
            process_Bag_Box_data.pop(len(ca_more)-i-1)

    #放入数据
    head_Bag_Box_new=[y[0] for y in process_Bag_Box_data]
    head_Bag_Box_set = set(head_Bag_Box_new)

    #记录所用箱子
    flag_bag = 0
    name_list = []
    for i in head_Bag_Box_set:
        print("第%s个订单处理"%i)
        index = []
        input = []
        for j in range(len(process_Bag_Box_data)):
            if process_Bag_Box_data[j][0] == i:
                index.append(j)
        for k in index:
            input.append(process_Bag_Box_data[k])

        # 将数据导入到数组中
        print(input)
        while len(input) !=0:
            packer_Bag_Box = Packer()
            for g in range(len(input)):
                for m in range(input[g][4]):
                    packer_Bag_Box.add_item(Item('no%s'%g+'.%s'%m,input[g][1],input[g][2],input[g][3],0))

            packer_Bag_Box.add_bin(Bin('1', 179.3421433+x[0],119.3421433+x[1],71.6578567+x[2],10))
            packer_Bag_Box.add_bin(Bin('2', 210.12536361+x[3],150.12536361+x[4],90.87463639+x[5],8))
            packer_Bag_Box.add_bin(Bin('3', 280.66217398+x[6],220.66217398+x[7],120.33782602+x[8],15))
            packer_Bag_Box.add_bin(Bin('4', 305.92446463+x[9],245.92446463+x[10],145.07553537+x[11],23))
            packer_Bag_Box.add_bin(Bin('5', 165+x[12],120+x[13],55+x[14],45))
            packer_Bag_Box.add_bin(Bin('6', 200+x[15],140+x[16],70+x[17],67))
            packer_Bag_Box.add_bin(Bin('7', 200+x[18],150+x[19],150+x[20],103))
            packer_Bag_Box.add_bin(Bin('8', 270+x[21],200+x[22],90+x[23],132))
            packer_Bag_Box.add_bin(Bin('9', 300+x[24],200+x[25],170+x[26],179))
            k,name_add = packer_Bag_Box.pack()
            input = k.copy()
            del k
            flag_bag = flag_bag+1
            name_list.append(name_add)

            del packer_Bag_Box
            gc.collect()
    total_volume = 0
    for j in range(len(name_list)):
        total_volume = total_volume+Bag_volume[int(name_list[j])-1]

    return total_volume
    #返回耗材使用容积





In [ ]:
#针对优化问题，
import warnings
warnings.filterwarnings("ignore")


import numpy as np
import matplotlib.pyplot as plt
import matplotlib
# 设置字体和设置负号
matplotlib.rc("font", family="KaiTi")
matplotlib.rcParams["axes.unicode_minus"] = False

# 初始化种群，群体规模，每个粒子的速度和规模
N = 2 # 种群数目
D = 27 # 维度
T = 4 # 最大迭代次数
c1 = c2 = 1.5 # 个体学习因子与群体学习因子
w_max = 0.8 # 权重系数最大值
w_min = 0.4 # 权重系数最小值
x_max = 0 # 每个维度最大取值范围，如果每个维度不一样，那么可以写一个数组，下面代码依次需要改变
x_min = -30 # 同上
v_max = 10 # 每个维度粒子的最大速度
v_min = -10 # 每个维度粒子的最小速度

# 初始化种群个体
x = np.random.rand(N, D) * (x_max - x_min) + x_min # 初始化每个粒子的位置
v = np.random.rand(N, D) * (v_max - v_min) + v_min # 初始化每个粒子的速度

# 初始化个体最优位置和最优值
p = x # 用来存储每一个粒子的历史最优位置
p_best = np.ones((N, 1))  # 每行存储的是最优值,初始化都为1
for i in range(N): # 初始化每个粒子的最优值，此时就是把位置带进去，把适应度值计算出来
    p_best[i] = func(x[i, :])

# 初始化全局最优位置和全局最优值
g_best = float("inf") # 设置真的全局最优值
gb = np.ones(T) # 用于记录每一次迭代的全局最优值
x_best = np.ones(D) # 用于存储最优粒子的取值

# 按照公式依次迭代直到满足精度或者迭代次数
for i in range(T):
    for j in range(N):
        # 个更新个体最优值和全局最优值
        if p_best[j] > func(x[j,:]):
            p_best[j] = func(x[j,:])
            p[j,:] = x[j,:].copy()
        # p_best[j] = func(x[j,:]) if func(x[j,:]) < p_best[j] else p_best[j]
        # 更新全局最优值
        if g_best > p_best[j]:
            g_best = p_best[j]
            x_best = x[j,:].copy()   # 一定要加copy，否则后面x[j,:]更新也会将x_best更新
        # 计算动态惯性权重
        w = w_max - (w_max - w_min) * i / T
        # 更新位置和速度
        v[j, :] = w * v[j, :] + c1 * np.random.rand(1) * (p[j, :] - x[j, :]) + c2 * np.random.rand(1) * (x_best - x[j, :])
        x[j, :] = x[j, :] + v[j, :]
        # 边界条件处理
        for ii in range(D):
            if (v[j, ii] > v_max) or (v[j, ii] < v_min):
                v[j, ii] = v_min + np.random.rand(1) * (v_max - v_min)
            if (x[j, ii] > x_max) or (x[j, ii] < x_min):
                x[j, ii] = x_min + np.random.rand(1) * (x_max - x_min)
    # 记录历代全局最优值
    gb[i] = g_best
print("最优值为", gb[T - 1], "最优位置为", x_best)
plt.plot(range(T),gb)
plt.xlabel("迭代次数")
plt.ylabel("适应度值")
plt.title("适应度进化曲线")
plt.show()

In [2]:
x = list(range(1,51))
print(x)


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50]
